In [ ]:
# Install the following when running for first time 

#%%capture
#!pip install --upgrade pip
#!pip3 install seaborn
#!pip install spacy
#!pip3 install pyinflect
#!pip3 install lemminflect
#import lemminflect 
#!pip install graphviz
#!pip install pydotplus
#!pip install gensim python-Levenshtein wordcloud
#!pip install docx2txt
#!pip install gensim
#!pip install pyldavis --user
#!pip install pyldavis
#!pip install scikit-learn

In [1]:
import pyinflect
import spacy
import matplotlib.pyplot as plt
from spacy import displacy
import scipy.stats as stats
import numpy as np
import os
import pandas as pd
from pylab import rcParams
import re
import seaborn as sns
import nltk 
from nltk.corpus import wordnet 
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# import numpy for matrix operation
import numpy as np

# import LDA from sklearn
from sklearn.decomposition import LatentDirichletAllocation

import pydotplus
nltk.download(['wordnet', 'stopwords', 'punkt'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import docx2txt
# read in word file
doc = docx2txt.process("cambridge report.docx")

C:\Users\AI\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
tk=gensim.utils.simple_preprocess(doc, deacc=True)

In [4]:
corpus=tk

In [5]:
# Apply Preprocessing on the Corpus

# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized

# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus]   

In [22]:
# Converting text into numerical representation
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)

# Converting text into numerical representation
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
# Array from TF-IDF Vectorizer 
tf_idf_arr = tf_idf_vectorizer.fit_transform(clean_corpus)

# Array from Count Vectorizer 
cv_arr = cv_vectorizer.fit_transform(clean_corpus)
# this is our converted text to numerical representation from the Tf-IDF vectorizer
dtm=cv_arr
cv=cv_vectorizer

# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tf_idf_vectorizer.get_feature_names()

# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tf_idf_vectorizer.get_feature_names()

# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

In [26]:
# Implementation of LDA:
    
# Create object for the LDA class 
# Inside this class LDA: define the components:
lda_model = LatentDirichletAllocation(n_components = 6, max_iter = 20, random_state = 20)

lda_output = lda_model.fit_transform(dtm)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(tf_idf_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_

In [27]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['muslim' 'hatred' 'legislation' 'incitement']
Topic 2 ['offence' 'religion' 'government' 'act']
Topic 3 ['community' 'law' 'provision' 'right']
Topic 4 ['however' 'criminal' 'could' 'blasphemy']
Topic 5 ['christian' 'new' 'ground' 'house']
Topic 6 ['religious' 'bill' 'faith' 'would']


In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0       1.465019 -47.625065       1        1  18.819673
1      54.854797 -36.476795       2        1  16.893911
2      32.212929 -61.444572       3        1  16.570618
4       5.111096 -14.110332       4        1  16.036456
5      26.350170 -33.374702       5        1  15.924014
3      38.109818  -7.222781       6        1  15.755328, topic_info=            Term       Freq      Total Category  logprob  loglift
752       muslim  60.000000  60.000000  Default  30.0000  30.0000
963    religious  53.000000  53.000000  Default  29.0000  29.0000
220    community  38.000000  38.000000  Default  28.0000  28.0000
650          law  35.000000  35.000000  Default  27.0000  27.0000
790      offence  33.000000  33.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
747    motivated   3.926065   4.540156   Topic6  -4.9610   1.7027
809        order   3.926065   4.540156   Topic6  -4.9610   1.7027
593    insulting   3.926065   4.540156   Topic6  -4.9610   1.7027
500      general   3.926065   4.540156   Topic6  -4.9610   1.7027
912  publication   3.166178   3.780270   Topic6  -5.1762   1.6707

[210 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5         4  0.887735       abuse
7         2  0.906708     abusive
18        2  0.990206         act
25        4  0.887735  additional
30        2  0.971679    adherent
...     ...       ...         ...
1236      4  0.931105      whilst
1237      1  0.809015       whole
1243      5  0.797104      within
1245      4  0.831684        word
1251      5  0.978886       would

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 5, 6, 4])

In [29]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, dtm, cv, mds='mmds')
panel

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.111118  0.284174       1        1  18.819673
1     -0.082007 -0.251527       2        1  16.893911
2     -0.262372 -0.050785       3        1  16.570618
4     -0.158811  0.178029       4        1  16.036456
5      0.182984 -0.193996       5        1  15.924014
3      0.209088  0.034105       6        1  15.755328, topic_info=            Term       Freq      Total Category  logprob  loglift
752       muslim  60.000000  60.000000  Default  30.0000  30.0000
963    religious  53.000000  53.000000  Default  29.0000  29.0000
220    community  38.000000  38.000000  Default  28.0000  28.0000
650          law  35.000000  35.000000  Default  27.0000  27.0000
790      offence  33.000000  33.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
747    motivated   3.926065   4.540156   Topic6  -4.9610   1.7027
809        order   3.926065   4.540156   Topic6  -4.9610   1.7027
593    insulting   3.926065   4.540156   Topic6  -4.9610   1.7027
500      general   3.926065   4.540156   Topic6  -4.9610   1.7027
912  publication   3.166178   3.780270   Topic6  -5.1762   1.6707

[210 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5         4  0.887735       abuse
7         2  0.906708     abusive
18        2  0.990206         act
25        4  0.887735  additional
30        2  0.971679    adherent
...     ...       ...         ...
1236      4  0.931105      whilst
1237      1  0.809015       whole
1243      5  0.797104      within
1245      4  0.831684        word
1251      5  0.978886       would

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 5, 6, 4])

In [25]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['muslim' 'hatred' 'legislation' 'british']
Topic 2 ['religion' 'right' 'could' 'home']
Topic 3 ['community' 'incitement' 'however' 'belief']
Topic 4 ['protection' 'faith' 'would' 'lord']
Topic 5 ['religious' 'offence' 'racial' 'group']
Topic 6 ['law' 'government' 'bill' 'crime']
